# Course 4: Planning
## Part 2: Trajectory Generation
#### By Jonathan L. Moran (jonathan.moran107@gmail.com)
From the Self-Driving Car Engineer Nanodegree programme offered at Udacity.

## Objectives

* Form a strong understanding of the basics of motion planning algorithms;
* Review the hybrid [A*](https://en.wikipedia.org/wiki/A*_search_algorithm) and its use in the Stanford Junior;
* Implement the hybrid [A*](https://en.wikipedia.org/wiki/A*_search_algorithm) search algorithm in Python and C++.

## 1. Introduction

### 1.1. Types of Motion Planning Algorithms

#### Background

In this lesson, we cover the basics of [motion planning](https://en.wikipedia.org/wiki/Motion_planning) algorithms; here we move from behaviour selection to full trajectory generation. In order to produce not only trajectory curves but also the time-sequence at which a robot executes these curves, we employ a variety of motion planning algorithms and direct our attention towards the areas of safety, feasibility, comfort and efficiency while generating vehicle trajectories.

#### Properties and problem set-up

Several important properties of motion planning algorithms are _completeness_ and _optimality_. The _completness_ assumption asserts that if a solution exists, the motion planner is guaranteed to always find the solution. In scenarios where no solution exists, the algorithm will correctly report so.  The _optimality_ assumption asserts that, for a given cost function evaluating sequences of actions, the motion planner is guaranteed to always return a feasible (i.e., executable) sequence of actions with minimal cost. Several other assumptions exist for specific algorithms, namely the resolution completeness and probabilistic completeness in the case of sampling-based planning algorithms.  

In order to enforce these constraints, we need a well-defined problem — the [_configuration space_](https://en.wikipedia.org/wiki/Configuration_space_\(physics\)) defines all possible configurations a robot can make in the world coordinate frame. In a 2D environment, this corresponds to each and every position $\left[x, y\right]$. In 3D, we extend the 2D position to include an orientation angle $\left[x, y, \theta\right]$. We must also define a _start_ and _goal configuration_; the start configuration $q_{\mathrm{start}}$ is usually initialised from a localisation algorithm using sensor data, whereas the goal configuration $q_{\mathrm{goal}}$ is selected w.r.t. the behaviour planner defining where to move and at which speed. In practise, motion planning problem definitions usually consider additional constraints, such as the sequence of actions predicted for non-vehicles and pedestrians, which can be obtained via map and traffic data as well as kinematic / physics-based models.

#### Types of motion planning algorithms

Several well-known types motion planning algorithms are as follows:
* [Combinatorial methods](http://msl.cs.uiuc.edu/planning/ch6.pdf): combinatorial approaches to motion planning find paths through the continuous configuration space without resorting to approximations — for special classes such as 2D robot planning, combinatorial methods provide _complete_, efficient, approximation-independent solutions over sampling-based planning methods [1];
    * Relies on a _roadmap_ assumption through cell decomposition (i.e., partitioning the search space into cells) and several properties regarding accessibility and connectivity of the discrete graph search sapce; 
* [Artificial potential fields](https://en.wikipedia.org/wiki/Motion_planning#Artificial_potential_fields): this type of reactive method defines obstacles as electrostatic point charges which generate potential fields — the robot navigates the anti-gravity field around e.g., pedestrians, bicyclists, using a [navigation function](https://en.wikipedia.org/wiki/Navigation_function).
    * A disadvantage with this class of algorithms (aside from harmonic potential field-based implementations) is their sensitivity to local minima, which might prevent the planner from arriving at the intended goal state;
* [Optimal control](https://en.wikipedia.org/wiki/Optimal_control): a set of methods for [trajectory optimisation](https://en.wikipedia.org/wiki/Trajectory_optimization) using either [dynamic programming](https://en.wikipedia.org/wiki/Dynamic_programming) and principle of optimality assumption (in closed-loop), or [calculus of variations](https://en.wikipedia.org/wiki/Calculus_of_variations) (in open-loop);
    * Solves both the motion planning and control input generation problems in one approach using dynamic vehicle model with a start- and end configuration;
    * Here a sequence of control inputs (e.g., steering angle, throttle) are generated and optimised with a cost function relative to the configuration of the vehicle (e.g., maximising efficiency, maintaining safe distance to obstacles);
    * Optimal control relies on numerical approximation methods, or [reinforcement learning](https://en.wikipedia.org/wiki/Reinforcement_learning)-based approaches for closed-loop control;
* [Sampling-based methods](https://en.wikipedia.org/wiki/Motion_planning#Sampling-based_algorithms): unlike combinatorial approaches, these set of algorithms have a runtime that is not explicitly exponentially dependent on the dimensionality of the configuration space $C$ — sampling-based algorithms construct roadmaps using a less-elegant collision dectection test which is _probabilistically complete_. In other words, the probability of a solution approaches $1.0$ as the free search space is exahusted;
    * For a cell decomposition / discretisation approach, [graph-based search](https://en.wikipedia.org/wiki/Graph_traversal) algorithms (e.g., [A*](https://en.wikipedia.org/wiki/A*_search_algorithm), [Dijkstra's](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm), [D* / D* Lite](https://en.wikipedia.org/wiki/D*), [ARA*](https://en.wikipedia.org/wiki/Anytime_A*), etc.);
    * For probabilistc methods, i.e., random exploration of configuration and input space, [probabilistic data structures](https://en.wikipedia.org/wiki/Category:Probabilistic_data_structures) are used (e.g., [RRT / RRT*](https://en.wikipedia.org/wiki/Rapidly-exploring_random_tree#Variants), [PRM](https://en.wikipedia.org/wiki/Probabilistic_roadmap), etc.).

### 1.2. Hybrid A*

#### Background

Hybrid A* (also known as Hybrid-State A* Search [2]), is a path-planning algorithm adapting the well-known [A* search](https://en.wikipedia.org/wiki/A*_search_algorithm) to 3D kinematic state space of the vehicle. The Hybrid A* search algorithm uses a modified state-update rule to capture the continuous state of the vehicle in the discretised map space (represented as nodes in the A* graph). This guarantees kinematic feasability of the generated trajectory. In other words, Hybrid A* finds a path that a vehicle can actually manoeuvre (e.g., smooth curve-based trajectories) which is favoured by vehicle motion models such as the [kinematic bicycle](https://thomasfermi.github.io/Algorithms-for-Automated-Driving/Control/BicycleModel.html).

#### Applications in path planning

Unlike the standard A* algorithm, the Hybrid A* is *not* guaranteed to find an minimal-cost solution. It is not _complete_, meaning it does not always find a solution when one exists. Aside from this, Hybrid A* has been implemented in Stanford's _Junior_ [3], the [2007 DARPA Grand Challenge](https://en.wikipedia.org/wiki/DARPA_Grand_Challenge_\(2007\)) 2nd Place winner, to perform free-form navigation in abritrary environments (e.g., parking lots). In order to perform trajectory planning, the Hybrid A* algorithm explores / expands discrete states using continuous vehicle coordinates $<x, y, \theta>$. A prediction $<x^{\prime}, y^{\prime}, \theta^{\prime}$ is made with respect to the vehicle heading, and the path selected by Hybrid A* is guided by two additional heuristics: _non-holonomic-without-obstacles_, and the _holonomic-with-obstacles_. In addition to the runtime-, behaviour-, manoeuvre execution- and obstacle distance-based costs of the standard A* search, the Stanford Junior team used these two differential constraints to account for the non-holonomic nature of the ego-vehicle w.r.t. its location, orientation and direction of motion. This heuristic aided Junior in approaching the goal state with the desired heading. The second differential motion heuristic, _holonomic-with-obstacles_, ignores the non-holonomic nature of the vehicle and emphasizes the shortest distance to the goal. This second heuristic is computed online via a dynamic programming algorithm in 2D, whereas the first heuristic is computed entirely offline for the entire 4D space (vehicle location, orientation, and direction of motion). The _non-holonomic-without-obstacles_ heuristic is said to be significantly more efficient than Euclidean distance as it takes into account vehicle orientation, however, this heuristic alone fails to produce executable trajectories in situations with U-shaped dead ends. The addition of the _holonomic-with-obstacles_ heuristic attempts to address this by discovering these obstacles and dead-ends in 2D, which is used to guide the more-expensive 3D search away from these areas [2].

The Hybrid A* search uses analytic expansions based on the [Reed-Shepp model](http://lavalle.pl/planning/node822.html). This addresses the precision issue resulting from the discretisation of the vehicle control action space. The effect of steering angle discretisation on precision means that the grid resolution must be carefully selected such that a decent approximation of the exact continuous-coordinate goal state can be reached. A node in the tree is expanded by simulating a kinematic model of the car for a given control action over the time-step equal to the resolution of the grid. Then, for some nodes, the optimal Reed-Shepp path is computed from the current- to goal-state (assuming no obstacle-free environment). By selectively applying the Reed-Shepp expansion, this extension of the search tree yields signficant benefits in both planning time and accuracy [2]. Several other path-cost ([Voronoi Field](https://en.wikipedia.org/wiki/Voronoi_diagram#Engineering)) and local optimisation / smoothing functions are applied to the final path planner used in the Stanford _Junior_. 

#### Implementation in Python

The Python implementation below outlines the [A* search]() algorithm using the [kinematic bicycle motion] model (i.e., the hybrid A* algorithm). The following variables and objects are used:
* `State(x, y, theta, g, f)`: An object which stores the `x`, `y`, position coordinates, heading angle `theta` (direction) and the current A* `g` and `f` values;
* `grid`: A binary-valued 2D array — here a value "1" indicates the presence of an obstacle, wheras a "0" corresponds to free (navigable) space;
* `SPEED`: The speed of the vehicle used in the bicycle motion model;
* `LENGTH`: The length of the vehicle used in the bicycle motion model;
* `NUM_THETA_CELLS`: The number of cells to divide a circle into — used to keep track of which states have already been visited. 

The bulk of the hybrid A* algorithm is defined within the `search` function. The `expand` function takes a `State` instance and a `goal` (not implemented below), and returns a list of possible next-states for a range of steering angles. This function contains the implementation of the bicycle motion model and the A* heuristic function.

```python
def expand(
        state, 
        goal
):
    """Selects a trajectory using the bicycle motion model and heuristic function.
    
    :param state: Current position (`x`, `y`), orientation `theta` and heuristic `g`, `f` values.
    :param goal: Coordinates of the goal location. TODO.
    """
    
    next_states = []
    # Span the next-state space given by the vehicle heading (direction)
    for delta in range(-35, 40, 5):
        ### Begin the bicycle motion model
        delta_rad = deg_to_rad(delta)
        omega = SPEED / LENGTH * tan(delta_rad)
        next_x = state.x + SPEED * cos(theta)
        next_y = state.y + SPEED * sin(theta)
        next_theta = normalize(state.theta + omega)
        ### Compute the heuristic evaluation
        next_g += 1
        next_f += heuristic(next_x, next_y, goal)
        ### Create a next-state object
        state = State(next_x, next_y, next_theta, next_g, next_f)
        next_states.append(state)
    return next_states
    
def search(
        grid, 
        start, 
        goal
):
    """Performs the A* search with closing and opening / expansion of next-states.
    
    The vehicle heading (theta) search space is discretised, i.e., the heading
    interval [0, +/- 2*pi] is partitioned into `NUM_THETA_CELLS` such that the
    lower discretised `theta` values are assigned lower stack numbers, whereas
    the `higher` theta values closer to 2*pi are assigned to larger stack numbers.
    
    In order to perform A* search, we sort the remaining states in `states_opened`
    by lowest f-value; this is crucial to A* as the lowest f-value state indicates
    an efficient starting point for the search.
    
    :param grid: 2D discretised map of cells. TODO.
    :param start: State object to start the search from (i.e., 2D position and heading).
    :param goal: Coordinates of the goal location. TODO.
    """
    
    # Store the `State` instances to explore
    states_opened = []
    # Initialise the 3D map from the 2D grid and discretised heading angle
    states_closed = [
        [[0 for x in range(grid[0])] for y in range(len(grid))]
            for cell in range(NUM_THETA_CELLS)
    ]
    # Store the previous visited `State` instances
    states_came_from = [
        [[0 for x in range(grid[0])] for y in range(len(grid))]
        for cell in range(NUM_THETA_CELLS)
    ]
    # Create a new starting `State`
    x = start.x
    y = start.y
    theta = start.theta
    g = 0
    f = heuristic(start.x, start.y, goal)
    state = State(x, y, theta, g, f)
    states_opened.append(state)
    ### Discretising the theta search space
    stack_num = theta_to_stack_number(state.theta)
    states_closed[stack_num][idx(state.x)][idx(state.y)] = state
    ### Explore the remaining states in `states_opened`
    while states_opened:
        # First sort states by lowest f-value in order to start the search
        # Then, get the state with the lowest f-value
        states_opened.sort(key=lambda state: state.f)
        current = states_opened.pop(0)
        # Check if the goal has been reached
        if (
            (idx(current.x) == goal.x)
            and (idx(current.y) == goal.y)
        ):
            return path
       # Get the set of next-possible states
       next_states = expand(current, goal)
       for next_s in next_states:
           if not in_grid(next_s):
               # Skip any states not in grid dimensions
               continue
           # Check that this state has not previously been visited
           # and that no obstacle exists at this location
           stack_num = theta_to_stack_number(next_s.theta)
           if (
               (not states_closed[stack_num][idx(next_s.x)][idx(next_s.y)])
               and (idx(current.y) == goal.y)
           ):
               # Add the valid state to the explored states stack
               states_oppened.append(next_s)
               # Close the visited `State` tuple
               states_closed[stack_num][idx(next_s.x)][idx(next_s.y)] = 1
               # Add the visited state to the trajectory history
               states_came_from[stack_num][idx(next_s.x)][idx(next_s.y)] = current
 ```    

In the above Python code we have a near-complete implementation of the hybrid A* search, but note that the following functions are assumed to have been implemented outside of the provided code:
* `deg_to_rad`: Returns the heading angle converted from degrees to radians;
* `normalize()`: Returns the normalised vehicle heading (plus a shift amount `omega`);
* `in_grid`: Returns the boolean indicating whether the given state is within the dimensions of the discretised map;
* `index()`: Returns the corresponding index of the position coordinate in the grid space;
* `theta_to_stack_number()`: Returns the corresponding discretised theta angle from range [0, 2*pi] to stack space.

With the variables defined above the Python code-block, we have a complete definition of the hybrid A* search that we can use in Sect. 2.1. to implement in C++ code. 

## 2. Programming Task

### 2.1. Hybrid A* in C++

In [ ]:
// From J. Moran's `1_hybrid_breadth_first.cpp`

TODO.

## Credits

This lesson has been prepared by Sebastian Thrun and Emmanuel Boidot, 2021 (link [here](https://www.udacity.com/course/self-driving-car-engineer-nanodegree--nd0013)).

References
* [1] LaValle, S. M. Combinatorial Motion Planning. In: Planning Algorithms. Cambridge University Press. 2006. pp.206-256. [doi:10.1017/CBO9780511546877.008]().ISBN-13: 978-0521862059.
* [2] Dolgov, D. et al. Practical Search Techniques in Path Planning for Autonomous Driving. Association for the Advancement of Artificial Intelligence. Pre-print. 2008. [https://www.aaai.org/Library/Workshops/2008/ws08-10-006.php](https://www.aaai.org/Papers/Workshops/2008/WS-08-10/WS08-10-006.pdf).
* [3] Montemerlo, M. et al. Junior: The Stanford Entry in the Urban Challenge. Journal of Field Robotics. 2008. 25(9):569-597. [doi:10.1002/rob.20258](https://doi.org/10.1002/rob.20258).

Helpful resources:
* [Introduction to Optimal Control by Sumeet Singh | Princeton University ORF523 Lecture Notes](https://www.princeton.edu/~aaa/Public/Teaching/ORF523/ORF523_S21_Guest_Lecture.pdf)